In [1]:
import requests
import re
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import sys
import matplotlib.pyplot as plt
from collections import Counter
import xml.etree.ElementTree as ET
from typing import List, Dict, Any, Optional
import duckdb

In [12]:
entries_df = pd.read_parquet("../data/ruland-dictionaries.parquet")
len(entries_df)

3164

In [18]:
entries_df["target_len"] = entries_df["target_lemma"].str.split().str.len()
entries_df = entries_df[entries_df["target_len"] < 4].copy()
len(entries_df)

2845

In [20]:
entries_df.sample(10, random_state=0)

,ID,Type,XML_ID,Lemma,Phrase,Variants,Notes,Senses,SenseDef,SenseRaw,Translations,EntryLevelTranslations,target_lemma,target_len
1830,Ruland1612-Lapidis-nigredo,L,None,Lapidis nigredo,None,[],[],[{'def': 'die Schwaͤrtze deß Steins. Wann du d...,[die Schwaͤrtze deß Steins. Wann du das Fewer ...,[die Schwaͤrtze deß Steins. Wann du das Fewer ...,[[{'quote': 'die Schwaͤrtze deß Steins. Wann d...,[],lapis nigredo,2
3003,Ruland1612-Truphat,T,None,Truphat,None,[],[],"[{'def': 'est multa virtus minerarum, quae met...","[est multa virtus minerarum, quae metallum quo...","[est multa virtus minerarum, quae metallum quo...",[[]],[],truphat,1
253,"Ruland1612-Almarcab,-Almarchaz",A,None,"Almarcab, Almarchaz",None,[],[],"[{'def': 'id est, lytargirium', 'index': 1, 'r...","[id est, lytargirium]","[id est, lytargirium Glette.]","[[{'quote': 'Glette.', 'type': 'translation', ...",[],almarcab almarchaz,2
136,Ruland1612-Agar,A,None,Agar,None,"[Algit, Algerit]",[],"[{'def': 'i.e. id est calx', 'index': 1, 'raw'...",[i.e. id est calx],"[, i.e. id est calx, Algit, Algerit idem. Agab...","[[{'quote': 'Saltz', 'type': 'translation', 'x...",[],agar,1
2776,Ruland1612-Specificum,S,None,Specificum,None,[],[],"[{'def': None, 'index': 1, 'raw': 'formale est...",[None],"[formale est, quod speciem per formales propri...",[[]],[],specificus,1
1225,Ruland1612-Elanula,E,None,Elanula,None,[],[],"[{'def': 'Ist hartalumen, wie Eisen.', 'index'...","[Ist hartalumen, wie Eisen.]","[Ist hartalumen, wie Eisen.]","[[{'quote': 'Ist hartalumen, wie Eisen.', 'typ...",[],elanula,1
1152,Ruland1612-Diaphanum,D,None,Diaphanum,None,[],[],"[{'def': 'est id, quod transparendo lucet', 'i...","[est id, quod transparendo lucet]","[est id, quod transparendo lucet, Durchscheing .]","[[{'quote': 'Durchscheing', 'type': 'translati...",[],diaphanus,1
1236,Ruland1612-Elersna,E,None,Elersna,None,[],[],[{'def': 'Die Handlungdeß Silbers oder ein Sil...,[Die Handlungdeß Silbers oder ein Silberader o...,[Die Handlungdeß Silbers oder ein Silberader o...,[[{'quote': 'Die Handlungdeß Silbers oder ein ...,[],elersna,1
1642,Ruland1612-Ignis-calor,I,None,Ignis calor,None,[],[],"[{'def': None, 'index': 1, 'raw': 'aut est nat...",[None],"[aut est naturalis, aut artificialis.]",[[]],[],ignis calor,2
218,Ruland1612-Algemet,A,None,Algemet,None,[],[],"[{'def': 'id est, Carbones.', 'index': 1, 'raw...","[id est, Carbones.]","[id est, Carbones.]",[[]],[],algemeo,1


In [21]:
entries_df[entries_df.duplicated(subset=["target_lemma"])]

,ID,Type,XML_ID,Lemma,Phrase,Variants,Notes,Senses,SenseDef,SenseRaw,Translations,EntryLevelTranslations,target_lemma,target_len
42,Ruland1612-Adamas,A,None,Adamas,None,[],"[Indica non nascens in auro, cognatus colore C...",[{'def': 'Demanth/Demuth nascitur & extra auru...,"[Demanth/Demuth nascitur & extra aurum, & in a...","[Demanth/Demuth nascitur & extra aurum, & in a...","[[{'quote': 'Demanth/Demuth', 'type': 'transla...",[],adamas,1
110,Ruland1612-Aeris,A,None,Aeris,None,[],[],"[{'def': ', id est, aerugo.', 'index': 1, 'raw...","[, id est, aerugo.]","[, id est, aerugo.]",[[]],[],aer,1
124,Ruland1612-Aetites,A,None,Aetites,None,[],"[Alii vero sex species Aetitis enumerant, quas...","[{'def': None, 'index': 1, 'raw': 'omnes conti...",[None],[omnes continent & custodiunt partus & valent ...,"[[{'quote': 'Ein Adlerstein', 'type': 'transla...",[],aetites,1
157,Ruland1612-Albani,A,None,Albani,None,[],[],"[{'def': None, 'index': 1, 'raw': 'lapis salis...",[None],[lapis salis lacti.],[[]],[],albanus,1
191,Ruland1612-Alcohol,A,None,Alcohol,None,[],[],"[{'def': ', id est, stybium siue antimonium.',...","[, id est, stybium siue antimonium.]","[, id est, stybium siue antimonium.]",[[]],[],alcohol,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,Ruland1612-Vena-cumulata,V,None,Vena cumulata,None,[],[],"[{'def': None, 'index': 1, 'raw': 'Stock', 'tr...",[None],[Stock],"[[{'quote': 'Stock', 'type': 'translation', 'x...",[],uena cumulo,2
3050,Ruland1612-Venam,V,None,Venam,None,[],[],"[{'def': None, 'index': 1, 'raw': 'qua parte a...",[None],"[qua parte abundat metallo, luto oblinire, Ver...","[[{'quote': 'Verstreichen das Ertz', 'type': '...",[],uena,1
3056,Ruland1612-Venus,V,None,Venus,None,[],[],"[{'def': None, 'index': 1, 'raw': 'der vnreine...",[None],"[der vnreine lapis, die Materi.]","[[{'quote': 'der vnreine lapis, die Materi.', ...",[],uenus,1
3110,Ruland1612-Vrina,V,None,Vrina,None,[],[],"[{'def': 'vini est acetum, aut vrina hominis v...","[vini est acetum, aut vrina hominis vinum perp...","[vini est acetum, aut vrina hominis vinum perp...","[[{'quote': 'Essig oder Harn eines Menschen, d...",[],urina,1


In [ ]:
entries_df = entries_df.drop_duplicates(subset=["target_lemma"])
le

In [3]:
emlap_metadata = pd.read_csv("https://raw.githubusercontent.com/CCS-ZCU/EMLAP_ETL/refs/heads/master/data/emlap_metadata.csv", sep=";")
emlap_metadata.head(5)

,working_title,No.,is_done,is_noscemus,if_noscemus_id,"#if is_noscemus = True, don't transcribe",AUTHORSHIP,is_one_author,#if more than 1 author skip section and choose compendium below,is_author_known,...,source_of_file,origin_of_copy,REFERENCES,catalogue_reference,secondary_references,general_comments,OTHER,filename,Unnamed: 64,tokens_N
0,"Augurello, Chrysopoeia",100001,True,True,713324.0,NaN,NaN,True,NaN,True,...,Noscemus,Unknown,NaN,Noscemus Wiki,Soranzo 2019,The 1518 Basel version is also in Noscemus,NaN,"Augurello,_Giovanni_Aurelio_-_Chrysopoeia__Ven...",NaN,23718
1,"Pseudo-Lull, Secretis",100002,True,False,NaN,NaN,NaN,True,NaN,True,...,MDZ,MBS,NaN,Hirsch 1950,NaN,"There is a prior, 1514 edition of De secretis ...",NaN,Pseudo-Lull1518_De_secretis_naturae_MDZ.pdf,NaN,24673
2,"Pantheus, Ars Transmutatione",100003,True,False,NaN,NaN,NaN,True,NaN,True,...,GB,BL,NaN,NaN,NaN,This book was first published in 1518 with an ...,NaN,Pantheus1518_Ars_Transmutationis_Metallicae_BL...,NaN,8646
3,"Pantheus, Commentarium",100004,True,False,NaN,NaN,NaN,True,NaN,True,...,MDZ,MSB,NaN,NaN,NaN,This 1519 book is catalogued wrongly by many l...,NaN,Pantheus1519_Commentarium_Transmutationis_Meta...,NaN,3521
4,"Pantheus, Voarchadumia",100005,True,False,NaN,NaN,NaN,True,NaN,True,...,ONB,ONB,NaN,NaN,NaN,Dedicated to Leonellus Marquis of Estense,NaN,Pantheus1530_Voarchadumia_ONB.pdf,NaN,20386


In [4]:
emlap_metadata.columns

Index(['working_title', 'No.', 'is_done', 'is_noscemus', 'if_noscemus_id',
       '#if is_noscemus = True, don't transcribe', 'AUTHORSHIP',
       'is_one_author',
       '#if more than 1 author skip section and choose compendium below',
       'is_author_known', 'author_name', 'author_name_alternatives',
       'author_viaf', 'author_cerl', 'author_comments', 'is_compendium',
       'if_compendium', 'TRANSLATION / EDITORSHIP', 'is_translator',
       'translator_name', 'translator_name_alternatives', 'translator_viaf',
       'translator_cerl', 'translator_comments', 'is_editor', 'editor_name',
       'editor_name_alternatives', 'editor_viaf', 'editor_cerl',
       'editor_comments', 'DATE OF PUBLICATION', 'date_publication',
       'date_certainty', 'date_not_after', 'date_not_before', 'date_comment',
       'TITLE', 'title_short', 'title_full', 'title_viaf', 'title_comments',
       'PLACE OF PUBLICATION', 'place_publication', 'place_geonames',
       'place_comments', 'PUBLISHER', 

In [5]:
conn = duckdb.connect('/srv/data/grela/grela_v0-4.duckdb', read_only=True)

In [7]:
print(entries_df["target_len"].value_counts())

target_len
1.0    1865
2.0     902
3.0      78
4.0      32
5.0      15
6.0       7
7.0       4
8.0       1
Name: count, dtype: int64


In [8]:
# Build a global order for ALL EMLAP tokens (with punctuation, for pretty KWIC)
conn.execute("""
CREATE OR REPLACE TEMP VIEW emlap_all_tokens AS
SELECT
  t.grela_id,
  t.sentence_id,
  s.position        AS sentence_position,
  t.token_id,
  t.token_text,
  LOWER(t.lemma)    AS lemma_lower,
  t.pos,
  t.char_start,
  t.char_end,
  ROW_NUMBER() OVER (
    PARTITION BY t.grela_id
    ORDER BY s.position, t.char_start
  ) AS global_token_pos
FROM tokens t
JOIN works w  ON t.grela_id = w.grela_id
JOIN sentences s USING (sentence_id)
WHERE w.grela_id LIKE 'emlap%';
""")

# A content-only stream for matching (no punctuation, lemma required), BUT keep the global pos
conn.execute("""
CREATE OR REPLACE TEMP VIEW emlap_content_tokens AS
SELECT
  a.*,
  ROW_NUMBER() OVER (
    PARTITION BY a.grela_id
    ORDER BY a.sentence_position, a.char_start
  ) AS global_content_pos
FROM emlap_all_tokens a
WHERE a.lemma_lower IS NOT NULL
  AND a.pos <> 'PUNCT';
""")

conn.execute("""
-- Full stream with a stable per-work order
CREATE OR REPLACE TEMP TABLE emlap_full_stream AS
SELECT
  t.grela_id,
  t.sentence_id,
  s.position AS sentence_position,
  t.token_id,
  t.token_text,
  LOWER(t.lemma) AS lemma_lower,
  t.pos,
  t.char_start,
  t.char_end,
  ROW_NUMBER() OVER (
    PARTITION BY t.grela_id
    ORDER BY s.position, t.char_start
  ) AS seq_full
FROM tokens t
JOIN works w  ON t.grela_id = w.grela_id
JOIN sentences s USING (sentence_id)
WHERE w.grela_id LIKE 'emlap%';

-- Content-only stream aligned to full stream with precomputed next hops/grams
CREATE OR REPLACE TEMP TABLE emlap_content_stream AS
WITH c AS (
  SELECT
    f.*,
    ROW_NUMBER() OVER (
      PARTITION BY f.grela_id
      ORDER BY f.sentence_position, f.char_start
    ) AS seq_content
  FROM emlap_full_stream f
  WHERE f.lemma_lower IS NOT NULL AND f.pos <> 'PUNCT'
)
SELECT
  c.*,
  LEAD(c.lemma_lower, 1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS l2,
  LEAD(c.lemma_lower, 2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS l3,
  LEAD(c.seq_full,     1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS next1_seq_full,
  LEAD(c.seq_full,     2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS next2_seq_full,
  -- optional prejoined strings to avoid CONCAT in the hot path
  CASE WHEN LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) IS NOT NULL
       THEN c.lemma_lower || ' ' || LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content)
  END AS n2,
  CASE WHEN LEAD(c.lemma_lower,2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) IS NOT NULL
       THEN c.lemma_lower || ' ' || LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) || ' ' ||
            LEAD(c.lemma_lower,2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content)
  END AS n3
FROM c;
""")

In [9]:
import re
import pandas as pd

def concordance_for_target_across_sentences(conn, target_lemma: str, window: int = 10) -> pd.DataFrame:
    """
    Cross-sentence KWIC in EMLAP with strict adjacency (no tokens—including punctuation—between
    matched tokens). Efficient via precomputed streams.

    Returns exactly:
      ['target_lemma','grela_id','target_sentence_id','start_sentence_id','end_sentence_id',
       'kwic_text','kwic_tokens','target_sentence_text','target_sentence_tokens']
    """
    # --- normalize inputs ---
    if not target_lemma or not isinstance(target_lemma, str):
        raise ValueError("target_lemma must be a non-empty string")
    tl = re.sub(r"\s+", " ", target_lemma.strip().lower())
    target_len = len(tl.split())
    if not (1 <= target_len <= 3):
        raise ValueError("Only 1–3-word targets supported (MAX_N=3).")

    # --- materialize once per session (idempotent) ---
    existing = set((conn.execute("""
        SELECT list(table_name) FROM information_schema.tables WHERE table_schema='temp'
    """).fetchone()[0] or []))

    if "emlap_full_stream" not in existing:
        conn.execute("""
        CREATE OR REPLACE TEMP TABLE emlap_full_stream AS
        SELECT
          t.grela_id,
          t.sentence_id,
          s.position AS sentence_position,
          t.token_id,
          t.token_text,
          LOWER(t.lemma) AS lemma_lower,
          t.pos,
          t.char_start,
          t.char_end,
          ROW_NUMBER() OVER (
            PARTITION BY t.grela_id
            ORDER BY s.position, t.char_start
          ) AS seq_full
        FROM tokens t
        JOIN works w  ON t.grela_id = w.grela_id
        JOIN sentences s USING (sentence_id)
        WHERE w.grela_id LIKE 'emlap%';
        """)
        conn.execute("CREATE INDEX IF NOT EXISTS idx_fs_seq ON emlap_full_stream(grela_id, seq_full);")

    if "emlap_content_stream" not in existing:
        conn.execute("""
        CREATE OR REPLACE TEMP TABLE emlap_content_stream AS
        WITH c AS (
          SELECT
            f.*,
            ROW_NUMBER() OVER (
              PARTITION BY f.grela_id
              ORDER BY f.sentence_position, f.char_start
            ) AS seq_content
          FROM emlap_full_stream f
          WHERE f.lemma_lower IS NOT NULL AND f.pos <> 'PUNCT'
        )
        SELECT
          c.*,
          LEAD(c.lemma_lower, 1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS l2,
          LEAD(c.lemma_lower, 2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS l3,
          LEAD(c.seq_full,     1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS next1_seq_full,
          LEAD(c.seq_full,     2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) AS next2_seq_full,
          CASE WHEN LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) IS NOT NULL
               THEN c.lemma_lower || ' ' || LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content)
          END AS n2,
          CASE WHEN LEAD(c.lemma_lower,2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) IS NOT NULL
               THEN c.lemma_lower || ' ' || LEAD(c.lemma_lower,1) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content) || ' ' ||
                    LEAD(c.lemma_lower,2) OVER (PARTITION BY c.grela_id ORDER BY c.seq_content)
          END AS n3
        FROM c;
        """)
        conn.execute("CREATE INDEX IF NOT EXISTS idx_cs_lemma ON emlap_content_stream(grela_id, lemma_lower);")

    # --- per-target query (strict adjacency; grouped per match via start_seq_full) ---
    sql = """
WITH raw_matches AS (
  SELECT
    cs.grela_id,
    cs.sentence_id AS target_sentence_id,
    cs.seq_full    AS start_seq_full,
    ?::INT         AS target_len
  FROM emlap_content_stream cs
  WHERE
    (? = 1 AND cs.lemma_lower = ?) OR
    (? = 2 AND cs.n2          = ?) OR
    (? = 3 AND cs.n3          = ?)
),
bounds AS (
  SELECT
    m.grela_id,
    m.target_sentence_id,
    m.start_seq_full,
    CASE m.target_len
      WHEN 1 THEN m.start_seq_full
      WHEN 2 THEN cs.next1_seq_full
      WHEN 3 THEN cs.next2_seq_full
    END AS end_seq_full
  FROM raw_matches m
  JOIN emlap_content_stream cs
    ON cs.grela_id = m.grela_id AND cs.seq_full = m.start_seq_full
  WHERE
    CASE m.target_len
      WHEN 1 THEN TRUE
      WHEN 2 THEN cs.next1_seq_full = m.start_seq_full + 1
      WHEN 3 THEN cs.next2_seq_full = m.start_seq_full + 2
    END
),
context AS (
  SELECT
    b.grela_id,
    b.target_sentence_id,
    b.start_seq_full,                      -- carry per-match key
    f.sentence_id,
    f.token_id,
    f.token_text,
    f.lemma_lower,
    f.pos,
    f.char_start,
    f.char_end,
    ROW_NUMBER() OVER (
      PARTITION BY b.grela_id, b.start_seq_full
      ORDER BY f.seq_full
    ) AS ord
  FROM bounds b
  JOIN emlap_full_stream f
    ON f.grela_id = b.grela_id
   AND f.seq_full BETWEEN (b.start_seq_full - ?) AND (b.end_seq_full + ?)
),
agg_kwic AS (
  SELECT
    grela_id,
    target_sentence_id,
    start_seq_full,                                        -- group per match
    LIST(sentence_id ORDER BY ord) AS window_sentence_ids, -- for start/end sentences
    STRING_AGG(token_text, ' ' ORDER BY ord) AS kwic_text,
    LIST(
      STRUCT_PACK(
        token_id := token_id,
        token_text := token_text,
        lemma := lemma_lower,
        pos := pos,
        sentence_id := sentence_id,
        char_start := char_start,
        char_end := char_end
      )
      ORDER BY ord
    ) AS kwic_tokens
  FROM context
  GROUP BY grela_id, target_sentence_id, start_seq_full
),
target_sentence_texts AS (
  SELECT
    grela_id,
    sentence_id,
    STRING_AGG(token_text, ' ' ORDER BY char_start) AS sentence_text
  FROM emlap_full_stream
  GROUP BY grela_id, sentence_id
),
target_sentence_tokens AS (
  SELECT
    grela_id,
    sentence_id,
    LIST(
      STRUCT_PACK(
        token_id := token_id,
        token_text := token_text,
        lemma := lemma_lower,
        pos := pos,
        char_start := char_start,
        char_end := char_end
      )
      ORDER BY char_start
    ) AS sentence_tokens
  FROM emlap_full_stream
  GROUP BY grela_id, sentence_id
)
SELECT
  ?::VARCHAR AS target_lemma,
  a.grela_id,
  a.target_sentence_id,
  a.window_sentence_ids[1]  AS start_sentence_id,
  a.window_sentence_ids[-1] AS end_sentence_id,
  a.kwic_text,
  a.kwic_tokens,
  tst.sentence_text      AS target_sentence_text,
  tstok.sentence_tokens  AS target_sentence_tokens
FROM agg_kwic a
LEFT JOIN target_sentence_texts  tst
  ON tst.grela_id = a.grela_id AND tst.sentence_id = a.target_sentence_id
LEFT JOIN target_sentence_tokens tstok
  ON tstok.grela_id = a.grela_id AND tstok.sentence_id = a.target_sentence_id
ORDER BY a.grela_id, a.target_sentence_id, a.start_seq_full;
"""
    params = [
        target_len,              # raw_matches: target_len
        target_len, tl,          # 1-gram selector
        target_len, tl,          # 2-gram selector
        target_len, tl,          # 3-gram selector
        window, window,          # KWIC window
        tl                       # echo back target_lemma
    ]
    return conn.execute(sql, params).fetch_df()

In [10]:
df = concordance_for_target_across_sentences(conn, "mercurius metallum", window=10)
df.sample(10, random_state=10)

,target_lemma,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens
7,mercurius metallum,emlap_100029,emlap_100029_457,emlap_100029_457,emlap_100029_458,"eorum , quae isti minoralia uocant , quintam e...","[{'token_id': 2007015, 'token_text': 'eorum', ...",Obseruandum igitur in Elixiris insequentibus s...,"[{'token_id': 2007009, 'token_text': 'Obseruan..."
21,mercurius metallum,emlap_100073,emlap_100073_685,emlap_100073_685,emlap_100073_686,"accipere ex mercurio & sulphure , Porro autem ...","[{'token_id': 642798, 'token_text': 'accipere'...",uidimus enim corpora in argentum uiuum conuers...,"[{'token_id': 642768, 'token_text': 'uidimus',..."
5,mercurius metallum,emlap_100022,emlap_100022_129,emlap_100022_129,emlap_100022_130,"omnibus moribus praedominatur ignis , & illud ...","[{'token_id': 3335488, 'token_text': 'omnibus'...","Uiuum causat metalla , quamuis adhuc bene diff...","[{'token_id': 3335441, 'token_text': 'Uiuum', ..."
2,mercurius metallum,emlap_100010,emlap_100010_504,emlap_100010_503,emlap_100010_504,"sulphure uerba faciemus . Prius diximus , per ...","[{'token_id': 164234, 'token_text': 'sulphure'...","Prius diximus , per calcinationem , mercurium ...","[{'token_id': 164238, 'token_text': 'Prius', '..."
13,mercurius metallum,emlap_100044,emlap_100044_3320,emlap_100044_3317,emlap_100044_3322,"Secundo . Mercurialis Seua , laxa Chimolea . T...","[{'token_id': 26691, 'token_text': 'Secundo', ...",Mercurius metallorum .,"[{'token_id': 26701, 'token_text': 'Mercurius'..."
19,mercurius metallum,emlap_100059,emlap_100059_915,emlap_100059_914,emlap_100059_915,est certa unctuositas educta ex metallis auri ...,"[{'token_id': 1317118, 'token_text': 'est', 'l...",Mercurius metallorum praecipitatus est reducti...,"[{'token_id': 1317128, 'token_text': 'Mercuriu..."
11,mercurius metallum,emlap_100038,emlap_100038_7937,emlap_100038_7937,emlap_100038_7938,"omnibus moribus praedominatur ignis , & illud ...","[{'token_id': 987574, 'token_text': 'omnibus',...","in qua omnibus moribus praedominatur ignis , &...","[{'token_id': 987572, 'token_text': 'in', 'lem..."
12,mercurius metallum,emlap_100038,emlap_100038_8181,emlap_100038_8181,emlap_100038_8181,", tunc sequeretur , quod de nouo extra primam ...","[{'token_id': 991994, 'token_text': ',', 'lemm...","Si esset possibile , tunc sequeretur , quod de...","[{'token_id': 991991, 'token_text': 'Si', 'lem..."
1,mercurius metallum,emlap_100010,emlap_100010_404,emlap_100010_404,emlap_100010_404,"similitudine uerae calcis communis , quia arge...","[{'token_id': 162382, 'token_text': 'similitud...",Compraehendis igitur ex supradictis rationibus...,"[{'token_id': 162364, 'token_text': 'Compraehe..."
22,mercurius metallum,emlap_100075,emlap_100075_1114,emlap_100075_1112,emlap_100075_1115,". Habet mercurialem seuam , & laxam chinoleam ...","[{'token_id': 2972018, 'token_text': '.', 'lem...","Continet mercurium metallorum praecipitatum , ...","[{'token_id': 2972027, 'token_text': 'Continet..."


In [11]:
df.to_dict("records")

[{'target_lemma': 'mercurius metallum',
  'grela_id': 'emlap_100007',
  'target_sentence_id': 'emlap_100007_1188',
  'start_sentence_id': 'emlap_100007_1188',
  'end_sentence_id': 'emlap_100007_1189',
  'kwic_text': 'omnibus moribus predominatur ignis , & illud dicitur elementum cum mercurio metallorum & c. sulphur patet supra . Quod Impossibile est minora',
  'kwic_tokens': array([{'token_id': 2096642, 'token_text': 'omnibus', 'lemma': 'omnis', 'pos': 'DET', 'sentence_id': 'emlap_100007_1188', 'char_start': 264, 'char_end': 271},
         {'token_id': 2096643, 'token_text': 'moribus', 'lemma': 'mos', 'pos': 'NOUN', 'sentence_id': 'emlap_100007_1188', 'char_start': 272, 'char_end': 279},
         {'token_id': 2096644, 'token_text': 'predominatur', 'lemma': 'predomino', 'pos': 'VERB', 'sentence_id': 'emlap_100007_1188', 'char_start': 280, 'char_end': 292},
         {'token_id': 2096645, 'token_text': 'ignis', 'lemma': 'ignis', 'pos': 'NOUN', 'sentence_id': 'emlap_100007_1188', 'char_sta

In [ ]:
entries_df["emlap_instances"] = entries_df["target_lemma"].apply(lambda x: concordance_for_target_across_sentences(conn, x, window=10)[["grela_id", "target_sentence_id", "target_sentence_text", "kwic_text"]].to_dict("records"))

In [50]:
entries_df["emlap_instances_N"] = entries_df["emlap_instances"].apply(len)

In [51]:
entries_df.sort_values("emlap_instances_N", ascending=False)[:20]

,ID,entry_xml,Type,XML_ID,Lemma,Phrase,Variants,Notes,Definitions,Translations,relemmatized,emlap_instances,instances_ids,emlap_instances_N
1620,Ruland1612-Hunc,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",H,None,Hunc,None,[],[],"[id est, Iupiter, stannum vel hucci.]",[],hic,"[(100001, 6, & Musis hanc commendauimus almis ...","[100001, 100001, 100001, 100001, 100001, 10000...",31023
396,Ruland1612-Aqua,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",A,None,Aqua,None,[],[1. Dieses Wasser reiniget/maschet/meitet/mach...,"[, id est, liquor, da...","[das Wasser ist auch immer darvonnen / da¬, Au...",aqua,"[(100001, 530, argento nec minus ipsi Uiuo eti...","[100001, 100001, 100001, 100001, 100001, 10000...",16821
1044,Ruland1612-Corpus,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",C,None,Corpus,None,[],[],[Clang. Buce. Der Coͤ...,[Der Coͤrper ist ein metallisch Wesen/dar¬],corpus,"[(100001, 101, & uasto uenas sub corpore condi...","[100001, 100001, 100001, 100001, 100001, 10000...",10750
1640,Ruland1612-Ignis,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",I,None,Ignis,None,[],[],[Ist nach etlicher Meinung das Oel sokauff der...,[Ist nach etlicher Meinung das Oel sokauff der...,ignis,"[(100001, 127, Tu molli e gremio surgens uxori...","[100001, 100001, 100001, 100001, 100001, 10000...",10030
1641,Ruland1612-Ignis,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",I,None,Ignis,None,[],[],[Ignis pro lapide philos. ...,[Bernhardus sagt: Das habe er],ignis,"[(100001, 127, Tu molli e gremio surgens uxori...","[100001, 100001, 100001, 100001, 100001, 10000...",10030
2498,Ruland1612-Rebus,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",R,None,Rebus,None,[],[],"[vltima rerum materia, ...",[die letzte Matery aller Ding],res,"[(100001, 4, Ut rerum inuolucris tantarum euol...","[100001, 100001, 100001, 100001, 100001, 10000...",8056
545,Ruland1612-Aurum,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",A,None,Aurum,None,[],"[Natiuum purum, quod a natura tale est, cuius ...","[, à Germanis Goldtsa...","[Goldtsa, Goldt ist der Coͤrper / ond ferment ...",aurum,"[(100001, 76, Interea certis hominum uis ulla ...","[100001, 100001, 100001, 100001, 100001, 10000...",6636
2222,Ruland1612-Oleum,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",O,None,Oleum,None,[],[],[est Destillatum aut Secretum.],[],oleum,"[(100001, 2245, Nunc oleo demissa leues:), (10...","[100001, 100002, 100002, 100002, 100002, 10000...",6292
2217,Ruland1612-Oleum,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",O,None,Oleum,None,[],[],[heist auch das erste Wass. ridas von sanguine...,[heist auch das erste Wass. ridas von sanguine...,oleum,"[(100001, 2245, Nunc oleo demissa leues:), (10...","[100001, 100002, 100002, 100002, 100002, 10000...",6292
2216,Ruland1612-Oleum,"[[[], [<Element '{http://www.tei-c.org/ns/1.0}...",O,None,Oleum,None,[],[],"[id est, ignis, wi...","[wirdt außgezogen von einer trocknen Sub¬, dis...",oleum,"[(100001, 2245, Nunc oleo demissa leues:), (10...","[100001, 100002, 100002, 100002, 100002, 10000...",6292


In [46]:
entries_df["instances_ids"] = entries_df["emlap_instances"].apply(lambda x: [ins[0] for ins in x])

In [90]:
entries_df.to_json("../data/large_files/ruland-emlap.json")